<a href="https://colab.research.google.com/github/changyuhsin1999/Nintendo_Switch_Game_Recommendation/blob/main/recommend_based_on_previous_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "Nintendo_Switch_Game_Recommendation" # Enter repo name
git_path = 'https://github.com/changyuhsin1999/Nintendo_Switch_Game_Recommendation.git'
!git clone "{git_path}"

rm: cannot remove './sample_data': No such file or directory
fatal: destination path 'Nintendo_Switch_Game_Recommendation' already exists and is not an empty directory.


In [41]:
import urllib
import zipfile
import time

import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
#Load and clean dataset
raw_df = pd.read_csv('/content/Nintendo_Switch_Game_Recommendation/switch-games.csv')
game_rating_df = raw_df.loc[:,['id','title','genre','rating']].dropna()
game_rating_df = game_rating_df[game_rating_df.rating != 0.0]
game_rating_df['id_num'] = pd.Series(game_rating_df['id']).astype('category')
game_rating_df['game_num'] = pd.Series(game_rating_df['title']).astype('category')
game_rating_df['id_num'] = game_rating_df['id_num'].cat.codes
game_rating_df['game_num'] = game_rating_df['game_num'].cat.codes
game_rating_df.head()

,id,title,genre,rating,id_num,game_num
3,e2f10932,Temtem,Action Adventure RPG,7.8,501,477
5,4377dbf1,Splatoon 3,Action,9.1,155,437
6,751a7bbf,JoJos Bizarre Adventure: All-Star Battle R,Fighting,7.0,267,206
8,438bc630,Pac-Man World Re-PAC,Action Adventure Platformer,6.9,157,335
9,7796d27e,Teenage Mutant Ninja Turtles: The Cowabunga Co...,Action Arcade Fighting Platformer,6.9,278,476


In [45]:
game_user_rating_pivot = game_rating_df.pivot(index = 'title',columns = 'id_num', values = 'rating').fillna(0)
game_user_rating_pivot.shape

(572, 572)

In [46]:
from scipy.sparse import csr_matrix
game_user_rating_matrix=csr_matrix(game_user_rating_pivot.values)
game_user_rating_matrix
game_user_rating_pivot

id_num,0,1,2,3,4,5,6,7,8,9,...,562,563,564,565,566,567,568,569,570,571
title,,,,,,,,,,,,,,,,,,,,,
1-2-Switch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Second Run Returns,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13 Sentinels: Aegis Rim,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88 Heroes: 98 Heroes Edition,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Hat In Time,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zombie Army 4: Dead War,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zombie Army Trilogy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zombieland: Double Tap - Road Trip,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric="cosine",algorithm="brute")
model_knn.fit(game_user_rating_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [50]:
#query_index=np.random.choice(us_canada_user_rating_pivot.shape[0])
query_index=68
#print(us_canada_user_rating_pivot.iloc[query_index,:])

distances,indices=model_knn.kneighbors(game_user_rating_pivot.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)
print("Distances --&gt;",distances," Indices --&gt;",indices)

print(distances.flatten())
print(len(distances.flatten()))

for i in range(0,len(distances.flatten())):
    if i==0:
        print("Recommendation for {0}:\n".format(game_user_rating_pivot.index[query_index]))
    else:
        print("{0}: {1}, with distance of {2}:".format(i,game_user_rating_pivot.index[indices.flatten()[i]],distances.flatten()[i]))

Distances --&gt; [[0. 1. 1. 1. 1. 1.]]  Indices --&gt; [[ 68 381 382 384 380 379]]
[0. 1. 1. 1. 1. 1.]
6
Recommendation for Call of Cthulhu:

1: Resident Evil Triple Pack, with distance of 1.0:
2: RiME, with distance of 1.0:
3: Ring Fit Adventure, with distance of 1.0:
4: Resident Evil Revelations Collection, with distance of 1.0:
5: Resident Evil Origins Collection, with distance of 1.0:
